In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def interpolacao_polinomial(x_pontos, y_pontos):
  """
  Calcula os coeficientes de um polinomio interpolador.
  A função gera e resolve o sistema linear Ac = y, onde A é a matriz de Vandermonde.

  Args:
    x_pontos (np.array): Coordenadas x dos pontos.
    y_pontos (np.array): Coordenadas y dos pontos.

  Returns:
    np.array: Vetor de coeficientes do polinômio [c0, c1, ..., cn-1]
  """

  n = len(x_pontos)
  # Gera a matriz de Vandermonde A
  A = np.zeros((n, n))
  for i in range(n):
    for j in range(n):
      A[i, j] = x_pontos[i]**j

  # Resolve um sistema linear Ac = y para encontrar os coeficientes c
  c = np.linalg.solve(A, y_pontos)

  return c

In [1]:
def avaliar_polinomio(coefs, x_valores):
  """Avalia o polinômio para um conjunto de valores x."""
  n = len(coefs)
  resultado = np.zeros_like(x_valores, dtype=float)
  for i in range(n):
    resultado += coefs[i] * (x_valores**i)
  return resultado

In [1]:
def spline_cubica_natural(x_pontos, y_pontos):
  """
  Calcula os coeficientes para uma spline cúbica natural.
    A função resolve um sistema linear tridiagonal para encontrar as segundas derivadas
    e então calcula os coeficientes de cada segmento cúbico.

    Args:
        x_pontos (np.array): Coordenadas x dos pontos.
        y_pontos (np.array): Coordenadas y dos pontos.

    Returns:
        np.array: Matriz de coeficientes (n-1 x 4) onde cada linha é [ai, bi, ci, di].
  """

  n = len(x_pontos)
  # h representa a distância entre os pontos x consecutivos
  h = np.diff(x_pontos)

  # Monta a matriz tridiagonal A para o sistema que resolve as segundas derivadas (M)
  A = np.zeros((n-2, n-2))
  for i in range(n-2):
    A[i, i] = 2 * (h[i] + h[i+1])
    if i < n - 3:
      A[i, i+1] = h[i+1]
      A[i+1, i] = h[i+1]

  # Monta o vetor B do sistema AM = B
  B = np.zeros(n - 2)
  for i in range(n - 2):
    B[i] = 6 * ((y_pontos[i+2] - y_pontos[i+1]) / h[i+1] - (y_pontos[i+1] - y_pontos[i]) / h[i])

  # Resolve o sistema para encontrar as segundas derivadas nos pontos internos
    M_internos = np.linalg.solve(A, B)

    # M completo, com M0 = Mn-1 = 0 para a spline natural
    M = np.concatenate(([0], M_internos, [0]))

    # Calcula os coeficientes de cada segmento da spline
    coefs = np.zeros((n - 1, 4))
    for i in range(n - 1):
        a_i = y_pontos[i] # a
        b_i = (y_pontos[i+1] - y_pontos[i]) / h[i] - h[i] * (2 * M[i] + M[i+1]) / 6 # b
        c_i = M[i] / 2 # c
        d_i = (M[i+1] - M[i]) / (6 * h[i]) # d
        coefs[i, :] = [a_i, b_i, c_i, d_i]

    return coefs